In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

cudnn.benchmark = True #-- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
                       #-- If this is set to false, uses some in-built heuristics that might not always be fastest.

cudnn.fastest = True #-- this is like the :fastest() mode for the Convolution modules,
                     #-- simply picks the fastest convolution algorithm, rather than tuning for workspace size

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim

import random, os, glob
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from models.alexnet import *
from utils import *

In [3]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config.ini")
# path to the dataset
path_to_GreenBackground = Config.get("Corpus", 'path_to_GreenBackground')

In [4]:
imSize = 225
batchSize = 32
nb_epoch = 50

In [5]:
# Load train and test data

userTestSet = ['C', 'D', 'E']
userTrainSet = ['F', 'G', 'H', 'I', 'J', 'K']

# Dictionary with the imgName as key and corresponding class as value for the test and train part
ImgNameToClassTrain = {}
for user in userTrainSet:
    for img in glob.glob(path_to_GreenBackground+user+'/img/*'):
        ImgNameToClassTrain[img]=img.split('_')[-2]
ImgNameToClassTest = {}
for user in userTestSet:
    for img in glob.glob(path_to_GreenBackground+user+'/img/*'):
        ImgNameToClassTest[img]=img.split('_')[-2]
# Convert the class to a numeric id
ClassToIdClass ={'1':0, '2':1, '3':2, '4':3, '5':4}        
# Associate imgName to the corresponding numeric class id
ImgNameToIdClassTrain = {ImgName:ClassToIdClass[ImgNameToClassTrain[ImgName]] for ImgName in  ImgNameToClassTrain.keys()}
ImgNameToIdClassTest = {ImgName:ClassToIdClass[ImgNameToClassTest[ImgName]] for ImgName in  ImgNameToClassTest.keys()}

In [9]:
dImgTrain = {imgName:Image.open(imgName) for imgName in ImgNameToClassTrain.keys()}
dImgTest = {imgName:Image.open(imgName).resize((imSize, imSize), Image.BILINEAR) for imgName in ImgNameToClassTest.keys()}

In [ ]:
# Compute mean and standard deviation for images normalisation
mean = ComputeMean(dImgTrain.values())
std = ComputeStdDev(dImgTrain.values())

imageTrainTransform = transforms.Compose([
    transforms.Scale(300), 
    transforms.RandomCrop(225), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

imageTestTransform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

In [ ]:
# define the criterion
criterion = nn.CrossEntropyLoss()

In [ ]:
# defined a new net with the number of classes corresponding to the dataset
alexTunedClassifier = alexnet(len(ImgNameToIdClassTrain)).train()
# load the pre-trained model and copy only the Features 
copyFeaturesParametersAlexnet(alexTunedClassifier, models.alexnet(pretrained=True))
alexTunedClassifier = alexTunedClassifier.cuda()

In [ ]:
#define the optimizer to train only the classifier part with lr of 0.01
optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.01, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

In [ ]:
# save the net
torch.save(alexTunedClassifier, "ckpt/gesture_alexFineTuned-50epoch-lr-0.01.ckpt")

In [ ]:
# load the model and define the optimizer to train only the classifier part with lr of 0.001
alexTunedClassifier = torch.load("ckpt/gesture_alexFineTuned-50epoch-lr-0.01.ckpt")
alexTunedClassifier = alexTunedClassifier.cuda()

optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

In [ ]:
# save the net
torch.save(alexTunedClassifier, "ckpt/gesture_alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")

In [ ]:
# load the model and define the optimizer to train net with lr of 0.01
alexTunedClassifier = torch.load("ckpt/gesture_alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")
alexTunedClassifier = alexTunedClassifier.cuda()

optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.001, momentum=0.9)
for epoch in range(50): # loop over the dataset multiple times
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

In [ ]:

# save the net
torch.save(alexTunedClassifier, "ckpt/gesture_alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.001.ckpt")

In [ ]:
# load the model and define the optimizer to train net with lr of 0.001
optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

In [ ]:
# save the net
torch.save(alexTunedClassifier, "ckpt/gesture_alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.001-50epoch-lr-0.0001.ckpt")